# BOPTEST Service APIs

This notebook introduces the BOPTEST-Service APIs, which will work with BOPTEST version 0.3:

These APIS are based on HTTP REST framework. 


## Authentication
We will soon begin optional authentication with the BOPTEST dashboard username. You can create a BOPTEST username in the dashboard.boptest.net website. This username will be sent along with the API calls in as the Authentication parameter in the API header. If you are authenticated, you will be able to track your test status on the dashboard. 



We start with a short setup that imports the requests and json Python modules. We also define the url to the BOPTEST server. We have set it to the BOPTEST API as default but feel free to change it to another server if you want to. 

In [52]:
import requests
import json

BOPTEST_URL = 'https://api.boptest.net'

def pretty_print_JSON(json_object):
  print(json.dumps(json_object, indent=2))

def print_BOPTEST_response(response):
  print(response['message'])

  if response['status'] == 200:
    payload = response['payload']
    print(f'The payload is {payload}')


## 1. The `/testcases` API:

The /testcases API tells you which testcases are already available for use on the BOPTEST server. This is an HTTP GET request which is prefixed by the BOPTEST server URL. 

In [2]:
all_testcases = requests.get(f"{BOPTEST_URL}/testcases")
pretty_print_JSON(all_testcases.json()[:6]) # Printing out the first six testcases

[
  {
    "testcaseid": "bestest_air"
  },
  {
    "testcaseid": "bestest_hydronic"
  },
  {
    "testcaseid": "bestest_hydronic_heat_pump"
  },
  {
    "testcaseid": "multizone_residential_hydronic"
  },
  {
    "testcaseid": "singlezone_commercial_hydronic"
  },
  {
    "testcaseid": "testcase1"
  }
]


# 2. GET `testcases/{testcase_name}` API

In [3]:
testcase_names = ["multizone_office_simple_air", "example_testcase_not_present"]

for testcase_name in testcase_names:
  testcase_presence = requests.get(f"{BOPTEST_URL}/testcases/{testcase_name}")
 
  if testcase_presence.status_code == 200:
    print(f"Yes! \"{testcase_name}\" is present!")
  else:
    print(f"Failed to confirm \"{testcase_name}\" presence with response code returning {testcase_presence.status_code} :\\")

Yes! "multizone_office_simple_air" is present!
Failed to confirm "example_testcase_not_present" presence with response code returning 404 :\


# 3. POST `testcases/{testcase_name}/select` API 
> AND
# 4. PUT `stop/{testid}`


We will go through both of these together. Since we want to `/stop` any tests we `/select` as soon as we think those tests are no longer necessary. 


## The `test_id`: 

The `test_id` is a unique identifier for each test. Most of the test specific BOPTEST API's require the `test_id` if you want to interact with the test. You get the `test_id` as a return value when you call the `/select` API.  

In [18]:
testcase_name = "bestest_hydronic_heat_pump"

test_id = requests.post(f"{BOPTEST_URL}/testcases/{testcase_name}/select").json()['testid']
print(f"The test id for this particular test is {test_id}.")

The test id for this particular test is e22b5a38-f27f-40f3-bceb-bf4b06a2413c.


## Getting to know your test:

### Scenarios:

### Test measurements:

### The control step:


This variable defines how much time the simulation will move forward at each step. 
There is a default value for the control step, so you don't have to set it if you don't want to. 


#### Getting the current/default control step: The GET `/step` API

This GET request will tell you what is the current control step is:

In [48]:
# Get default control step
response = requests.get(f'{BOPTEST_URL}/step/{test_id}').json()
print(response['message'])

if response['status'] == 200:
  step = response['payload']
  print(f'The default step period is {step}s.')


Queried the control step successfully.
The default step period is 1800s.


#### Setting your own control step: The PUT `/step` API

There is a PUT `/step` API which allows you to set your own control step in seconds:

In [46]:
# Set the control step
response = requests.put(f'{BOPTEST_URL}/step/{test_id}', data={'step':1800}).json()
print(response['message'])

if response['status'] == 200:
  step = response['payload']
  print(f'The control step period was set to {step}s.')


Control step set successfully.
The control step period was set to {'step': 1800}s.


In [47]:
# Check the control step now with the GET /step API:
response = requests.get(f'{BOPTEST_URL}/step/{test_id}').json()
print(response['message'])

if response['status'] == 200:
  step = response['payload']
  print(f'The control step period now is {step}s.')

Queried the control step successfully.
The control step period now is 1800s.




---



## Moving forward with the simulation: The `/advance` API:

You move the simulation forward using the `/advance` API. One call of the `/advance` API will move the simulation forward by one control step that we defined before. 

### Responses of the `/advance` API call:
The payload of the /advance call will show you the values of the current measurements. 

In [38]:
# A basic advance step
y = requests.post(f'{BOPTEST_URL}/advance/{test_id}').json()
print(y['message'])

if y['status'] == 200:
  print("The new measurements are", y['payload'])

Advanced simulation successfully from 9000.0s to 10800.0s.
The new measurements are {'ovePum_activate': 0, 'weaSta_reaWeaNOpa_y': 0.5, 'reaPFan_y': 510.20408163265313, 'oveTSet_u': 294.34999999999997, 'reaQHeaPumCon_y': 5770.2585306676065, 'reaTRet_y': 299.40617933272074, 'weaSta_reaWeaPAtm_y': 101325, 'weaSta_reaWeaTBlaSky_y': 258.292708653646, 'reaQHeaPumEva_y': -3742.9586342342636, 'weaSta_reaWeaNTot_y': 0.5, 'weaSta_reaWeaSolAlt_y': -0.906500406846257, 'reaTZon_y': 292.9681756591129, 'weaSta_reaWeaHHorIR_y': 252, 'weaSta_reaWeaSolTim_y': 8067.422175141299, 'oveHeaPumY_u': 0.5, 'weaSta_reaWeaCloTim_y': 10800, 'oveHeaPumY_activate': 1, 'reaPPumEmi_y': 20.498644281163177, 'weaSta_reaWeaHGloHor_y': 0, 'weaSta_reaWeaHDifHor_y': 0, 'oveTSet_activate': 0, 'weaSta_reaWeaRelHum_y': 0.6900000000000001, 'reaTSetHea_y': 294.15, 'reaCO2RooAir_y': 718.1218889169762, 'weaSta_reaWeaSolDec_y': -0.4027259272719727, 'ovePum_u': 1, 'reaPHeaPum_y': 2027.2998964915764, 'weaSta_reaWeaHDirNor_y': 0, 'reaT



---



## Interacting with your test:
## What inputs are available for the testcase you selected? : the GET `/inputs` API

The GET `/inputs` API will return what variables you can/have to set in the particular testcase you selected. 

The API return will tell you what are 
1. The minimum values for the variable.
2. A short description of the variable, 
3. The unit for the variable.
4. The maximum value for the variable. 

 

In [49]:
# Get inputs available
inputs = requests.get(f'{BOPTEST_URL}/inputs/{test_id}').json()
print(inputs['message'])

if inputs['status'] == 200:
  print("The inputs are", inputs['payload'])

Queried the inputs successfully.
The inputs are {'oveTSet_activate': {'Minimum': None, 'Description': 'Activation for Zone operative temperature setpoint', 'Unit': None, 'Maximum': None}, 'ovePum_activate': {'Minimum': None, 'Description': 'Activation for Integer signal to control the emission circuit pump either on or off', 'Unit': None, 'Maximum': None}, 'ovePum_u': {'Minimum': 0, 'Description': 'Integer signal to control the emission circuit pump either on or off', 'Unit': '1', 'Maximum': 1}, 'oveHeaPumY_u': {'Minimum': 0, 'Description': 'Heat pump modulating signal for compressor speed between 0 (not working) and 1 (working at maximum capacity)', 'Unit': '1', 'Maximum': 1}, 'oveTSet_u': {'Minimum': 278.15, 'Description': 'Zone operative temperature setpoint', 'Unit': 'K', 'Maximum': 308.15}, 'oveHeaPumY_activate': {'Minimum': None, 'Description': 'Activation for Heat pump modulating signal for compressor speed between 0 (not working) and 1 (working at maximum capacity)', 'Unit': No

### How to change the input values for the next control step:


The input variables can be set to desired values in a python dictionary.

In [26]:
# Example input dictionary
u = {'oveHeaPumY_u':0.5,
     'oveHeaPumY_activate': 1}

### Advancing with the new input values

The input dictionary be sent as an /advance call body. This will cause the simulation to advance by one step with the new inputs. 

In [27]:
# advance with u
y = requests.post(f'{BOPTEST_URL}/advance/{test_id}', data=u).json()

You will see the effects of this changed inputs in the response payload.

In [37]:
print(y['message'])
if y['status'] == 200:
  print("The new measurements are", y['payload'])

Advanced simulation successfully from 7200.0s to 9000.0s.
The new measurements are {'ovePum_activate': 0, 'weaSta_reaWeaNOpa_y': 0.275, 'reaPFan_y': 510.20408163265313, 'oveTSet_u': 294.34999999999997, 'reaQHeaPumCon_y': 6052.553419713639, 'reaTRet_y': 299.51334100258975, 'weaSta_reaWeaPAtm_y': 101325, 'weaSta_reaWeaTBlaSky_y': 258.74682385806346, 'reaQHeaPumEva_y': -4001.280588618656, 'weaSta_reaWeaNTot_y': 0.325, 'weaSta_reaWeaSolAlt_y': -0.9704608969219581, 'reaTZon_y': 292.9473181539115, 'weaSta_reaWeaHHorIR_y': 252.5625, 'weaSta_reaWeaSolTim_y': 6267.982794502697, 'oveHeaPumY_u': 0.5, 'weaSta_reaWeaCloTim_y': 9000, 'oveHeaPumY_activate': 1, 'reaPPumEmi_y': 20.498644281163177, 'weaSta_reaWeaHGloHor_y': 0, 'weaSta_reaWeaHDifHor_y': 0, 'oveTSet_activate': 0, 'weaSta_reaWeaRelHum_y': 0.6725, 'reaTSetHea_y': 294.15, 'reaCO2RooAir_y': 692.1705334189862, 'weaSta_reaWeaSolDec_y': -0.4027527643996826, 'ovePum_u': 1, 'reaPHeaPum_y': 2051.272831065368, 'weaSta_reaWeaHDirNor_y': 0, 'reaTSetCo



---



## Dynamically changing the inputs at every step: Make your own controller? 

A controller is an object that changes the input values for the next step based on the results from the last step. 

In the next cell you can see a basic controller implementation, which we have taken from <>.

In [66]:
class Controller_Proportional(object):
    
    def __init__(self, TSet=273.15+21, k_p=10.):
        '''Constructor.

        Parameters
        ----------
        TSet : float, optional
            Temperature set-point in Kelvin.
        k_p : float, optional
            Proportional gain. 
            
        '''
        
        self.TSet = TSet
        self.k_p  = k_p
    
    def compute_control(self, y):
        '''Compute the control input from the measurement.
    
        Parameters
        ----------
        y : dict
            Contains the current values of the measurements.
            {<measurement_name>:<measurement_value>}
    
        Returns
        -------
        u : dict
            Defines the control input to be used for the next step.
            {<input_name> : <input_value>}
    
        '''
    
        # Compute control
        if y['reaTZon_y']<self.TSet:
            e = self.TSet - y['reaTZon_y']
        else:
            e = 0
    
        value = self.k_p*e
        u = {'oveHeaPumY_u':value,
             'oveHeaPumY_activate': 1}
    
        return u

## Using your own controller with BOPTEST:

In [76]:
# Initialize scenario
y = requests.put(f'{BOPTEST_URL}/scenario/{test_id}', 
                 data={'time_period':'peak_heat_day',
                       'electricity_price':'dynamic'}).json()['payload']['time_period']

# Get the start time of the simulation
start_time_days = y['time']/24/3600
print(f"Start time of the simulation is {start_time_days} days")

# Set control step
requests.put(f'{BOPTEST_URL}/step/{test_id}', data={'step':3600})

# Instantiate controller
con = Controller_Proportional(TSet=273.15+21, k_p=5.)


Start time of the simulation is 16.0 days


In [77]:
# Simulation loop
from IPython.display import clear_output
while y:
    # Clear the display output at each step
    clear_output(wait=True)
    # Print the current operative temperature and simulation time
    print('-------------------------------------------------------------------')
    print('Operative temperature [degC]  = {:.2f}'.format(y['reaTZon_y']-273.15))
    simulation_time_days = y['time']/3600/24
    print('Simulation time [elapsed days] = {:.2f}'.format((simulation_time_days - \
                                                    start_time_days)))
    print('-------------------------------------------------------------------')
    # Compute control signal 
    u = con.compute_control(y)
    # Advance simulation with control signal
    y = requests.post(f'{BOPTEST_URL}/advance/{test_id}', data=u).json()['payload']

-------------------------------------------------------------------
Operative temperature [degC]  = 20.80
Simulation time [elapsed days] = 14.00
-------------------------------------------------------------------


## KPIs:

What are they?

How to get them?

In [51]:
response = requests.get(f'{BOPTEST_URL}/kpi/{test_id}').json()

print(response['message'])

if response['status'] == 200:
  payload = response['payload']
  print(f'The response payload is {payload}.')


Queried KPIs successfully.
The response payload is {'tdis_tot': 4.508723250280717, 'idis_tot': 0, 'ener_tot': 0.05004084973452596, 'cost_tot': 0.01268535540770233, 'emis_tot': 0.008356821905665834, 'pele_tot': 0.018718084575691674, 'pgas_tot': None, 'pdih_tot': None, 'time_rat': 0.09439101097318861}.


## Stopping the test:

### How to stop the test:


### Why it is important to stop tests? 
There are only a limited number of workers available for running tests. If all the available workers are busy, the next incoming test will be queued up till one of the workers get freed up. So it's up to you to be responsible for your own tests. Take note of your test_id, and stop the test as soon as you are done using it.  

In [83]:
stop_response = requests.put(f"{BOPTEST_URL}/stop/{test_id}")
if stop_response.status_code == 200:
  print("Successfully stopped the test!")
else:
  print(f"Couldn't stop test with status code: {stop_response.status_code}!")


Couldn't stop test with status code: 500!
